In [1]:
!pip install pandas statsmodels
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
!pip install arch
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 종속변수

In [2]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/insurance.csv').dropna()
df.columns.values[0] = 'date'
df_col = ['date', '변액종신', '변액연금', '변액유니버셜']
df = df[df_col]

df.head()

,date,변액종신,변액연금,변액유니버셜
0,2013_12,0.00768,0.01306,0.01242
1,2014_01,0.00765,0.01171,0.01152
2,2014_02,0.00910,0.01174,0.01193
3,2014_03,0.00904,0.01320,0.01275
4,2014_04,0.00901,0.01256,0.01186


## 기본통계량

In [3]:
df.describe()

,변액종신,변액연금,변액유니버셜
count,109.000000,109.000000,109.000000
mean,0.008250,0.010989,0.009709
std,0.004079,0.002462,0.002361
min,0.001460,0.000000,0.003280
25%,0.005670,0.009610,0.009010
50%,0.007330,0.011160,0.010150
75%,0.008510,0.012010,0.011040
max,0.026990,0.020720,0.018480


## 차분

In [4]:
df_col = ['변액종신', '변액연금', '변액유니버셜']

for col in df_col:
    df[f'{col}_1차차분'] = (df[col]).diff()
    df[f'{col}_2차차분'] = df[f'{col}_1차차분'].diff()

df.head(10)

,date,변액종신,변액연금,변액유니버셜,변액종신_1차차분,변액종신_2차차분,변액연금_1차차분,변액연금_2차차분,변액유니버셜_1차차분,변액유니버셜_2차차분
0,2013_12,0.00768,0.01306,0.01242,NaN,NaN,NaN,NaN,NaN,NaN
1,2014_01,0.00765,0.01171,0.01152,-0.00003,NaN,-0.00135,NaN,-0.00090,NaN
2,2014_02,0.00910,0.01174,0.01193,0.00145,0.00148,0.00003,0.00138,0.00041,0.00131
3,2014_03,0.00904,0.01320,0.01275,-0.00006,-0.00151,0.00146,0.00143,0.00082,0.00041
4,2014_04,0.00901,0.01256,0.01186,-0.00003,0.00003,-0.00064,-0.00210,-0.00089,-0.00171
5,2014_05,0.00746,0.01262,0.01206,-0.00155,-0.00152,0.00006,0.00070,0.00020,0.00109
6,2014_06,0.01042,0.01234,0.01205,0.00296,0.00451,-0.00028,-0.00034,-0.00001,-0.00021
7,2014_07,0.00740,0.01385,0.01289,-0.00302,-0.00598,0.00151,0.00179,0.00084,0.00085
8,2014_08,0.00885,0.01138,0.01157,0.00145,0.00447,-0.00247,-0.00398,-0.00132,-0.00216
9,2014_09,0.00881,0.01180,0.01133,-0.00004,-0.00149,0.00042,0.00289,-0.00024,0.00108


## 단위근 검정 : ADF, PP, KSPP

In [5]:
def adf_test(data):
    adf_result = adfuller(data.dropna())
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print('Critical Values:')
    for key, value in adf_result[4].items():
        print(f'   {key}: {value}')

def pp_test(data):
    pp_result = PhillipsPerron(data.dropna())
    print('\n PP Test Results:')
    print(f'   Test Statistic: {pp_result.stat}')
    print(f'   p-value: {pp_result.pvalue}')
    print(f'   Lags Used: {pp_result.lags}')

def kpss_test(data, regression='c'):
    kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)
    print('\n KPSS Test Results:')
    print(f'   KPSS Statistic: {kpss_stat}')
    print(f'   p-value: {p_value}')
    print(f'   Lags Used: {lags}')
    print('   Critical Values:')
    for key, value in critical_values.items():
        print(f'      {key}: {value}')

In [6]:
#변액종신
data = df['변액종신']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.2940372671255695
p-value: 0.015147121847537847
Critical Values:
   1%: -3.4936021509366793
   5%: -2.8892174239808703
   10%: -2.58153320754717

 PP Test Results:
   Test Statistic: -9.159673384586583
   p-value: 2.553119434652738e-15
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.28979767131958034
   p-value: 0.1
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [7]:
#변액종신
data = df['변액종신_1차차분']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.1174709172537165
p-value: 3.7978713626491704e-10
Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -32.01578266241821
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.152805370560023
   p-value: 0.1
   Lags Used: 33
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [8]:
#변액종신
data = df['변액종신_2차차분']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.20747527122827
p-value: 2.2802459072373866e-10
Critical Values:
   1%: -3.4996365338407074
   5%: -2.8918307730370025
   10%: -2.5829283377617176

 PP Test Results:
   Test Statistic: -76.24115747912376
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.12173124822151514
   p-value: 0.1
   Lags Used: 25
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [9]:
#변액연금
data = df['변액연금']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.517025360900343
p-value: 0.007560462240879886
Critical Values:
   1%: -3.4948504603223145
   5%: -2.889758398668639
   10%: -2.5818220155325444

 PP Test Results:
   Test Statistic: -9.836799027629642
   p-value: 4.871092765320639e-17
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.420006365780546
   p-value: 0.06853173888769569
   Lags Used: 3
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [10]:
#변액연금
data = df['변액연금_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -6.668894511718934
p-value: 4.641645206834342e-09
Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -33.01347282803619
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.148734969927696
   p-value: 0.1
   Lags Used: 25
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [11]:
#변액연금
data = df['변액연금_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.046443224509827
p-value: 1.79456030310871e-05
Critical Values:
   1%: -3.5003788874873405
   5%: -2.8921519665075235
   10%: -2.5830997960069446

 PP Test Results:
   Test Statistic: -76.05632401144888
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.13460506118589385
   p-value: 0.1
   Lags Used: 24
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [12]:
#변액유니버셜
data = df['변액유니버셜']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.266212403175712
p-value: 0.6444849923839199
Critical Values:
   1%: -3.4942202045135513
   5%: -2.889485291005291
   10%: -2.5816762131519275

 PP Test Results:
   Test Statistic: -8.433516820267922
   p-value: 1.8428025174187878e-13
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 1.178843002593408
   p-value: 0.01
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [13]:
#변액유니버셜
data = df['변액유니버셜_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.5125544119409615
p-value: 0.00767046368836602
Critical Values:
   1%: -3.5003788874873405
   5%: -2.8921519665075235
   10%: -2.5830997960069446

 PP Test Results:
   Test Statistic: -40.18434950736182
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.10425426745874752
   p-value: 0.1
   Lags Used: 21
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [14]:
#변액유니버셜
data = df['변액유니버셜_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.3456177621009004
p-value: 4.401885200823507e-06
Critical Values:
   1%: -3.5003788874873405
   5%: -2.8921519665075235
   10%: -2.5830997960069446

 PP Test Results:
   Test Statistic: -82.456073205811
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.1149483705280864
   p-value: 0.1
   Lags Used: 23
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [15]:
df_filtered = df[~df['date'].isin(['2018/10', '2018/11', '2018/12'])]
df_filtered[['date', '변액종신_2차차분', '변액연금_2차차분', '변액유니버셜_2차차분']].to_csv('종속변수.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('종속변수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 독립변수

In [16]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/독립변수.csv')

In [17]:
# 공백이나 특수문자 제거
df.columns = df.columns.str.strip()  # 양쪽 공백 제거
df.columns = df.columns.str.replace('(', '')  # 괄호 제거
df.columns = df.columns.str.replace(')', '')  # 괄호 제거

# 다시 열 이름 확인
print(df.columns)

Index(['date', 'CD91일', '실업률', '동행지수순환변동치', '총지수', 'KOSPI_증가'], dtype='object')


In [18]:
df.describe()

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
mean,45479.018182,1.665273,3.607273,99.807273,98.921227,2277.834455
std,106.676995,0.663282,0.393225,1.090384,4.081360,358.866290
min,45305.000000,0.630000,2.600000,96.400000,93.116000,1754.640000
25%,45396.250000,1.380000,3.500000,99.400000,95.402500,2012.552500
50%,45487.500000,1.620000,3.600000,99.900000,98.869500,2128.895000
75%,45579.750000,1.867500,3.800000,100.700000,100.187500,2441.227500
max,45648.000000,4.020000,4.800000,101.400000,109.260000,3296.680000


In [19]:
df['로그동행지수순환변동치'] = np.log(df['동행지수순환변동치'])
df['로그총지수'] = np.log(df['총지수'])
df['로그KOSPI_증가'] = np.log(df['KOSPI_증가'])

In [20]:
df_col = ['CD91일', '실업률', '로그동행지수순환변동치', '로그총지수', '로그KOSPI_증가']

for col in df_col:
    df[f'{col}_1차차분'] = df[col].diff()
    df[f'{col}_2차차분'] = df[f'{col}_1차차분'].diff()

df.head(10)

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가,로그동행지수순환변동치,로그총지수,로그KOSPI_증가,CD91일_1차차분,CD91일_2차차분,실업률_1차차분,실업률_2차차분,로그동행지수순환변동치_1차차분,로그동행지수순환변동치_2차차분,로그총지수_1차차분,로그총지수_2차차분,로그KOSPI_증가_1차차분,로그KOSPI_증가_2차차분
0,45609,2.65,3.1,100.0,93.116,2044.87,4.605170,4.533846,7.623089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45639,2.65,3.2,100.2,93.229,2011.34,4.607168,4.535059,7.606556,0.00,NaN,0.1,NaN,0.001998,NaN,0.001213,NaN,-0.016533,NaN
2,45305,2.65,3.2,100.5,93.730,1941.15,4.610158,4.540418,7.571036,0.00,0.00,0.0,-0.1,0.002990,0.000992,0.005359,0.004147,-0.035521,-0.018988
3,45336,2.65,3.6,100.5,93.980,1979.99,4.610158,4.543082,7.590847,0.00,0.00,0.4,0.4,0.000000,-0.002990,0.002664,-0.002696,0.019811,0.055332
4,45365,2.65,3.4,100.6,94.153,1985.61,4.611152,4.544921,7.593681,0.00,0.00,-0.2,-0.6,0.000995,0.000995,0.001839,-0.000825,0.002834,-0.016977
5,45396,2.65,3.6,100.4,94.214,1961.79,4.609162,4.545569,7.581613,0.00,0.00,0.2,0.4,-0.001990,-0.002985,0.000648,-0.001191,-0.012069,-0.014903
6,45426,2.65,3.5,100.0,94.369,1994.96,4.605170,4.547213,7.598379,0.00,0.00,-0.1,-0.3,-0.003992,-0.002002,0.001644,0.000996,0.016767,0.028836
7,45457,2.65,3.5,100.0,94.248,2002.21,4.605170,4.545930,7.602007,0.00,0.00,0.0,0.1,0.000000,0.003992,-0.001283,-0.002927,0.003628,-0.013139
8,45487,2.65,3.5,100.0,94.386,2076.12,4.605170,4.547393,7.638256,0.00,0.00,0.0,0.0,0.000000,0.000000,0.001463,0.002746,0.036249,0.032622
9,45518,2.52,3.6,100.0,94.551,2068.54,4.605170,4.549139,7.634598,-0.13,-0.13,0.1,0.1,0.000000,0.000000,0.001747,0.000283,-0.003658,-0.039907


In [21]:
data = df['CD91일']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.499294096443702
p-value: 0.5338605103596826
Critical Values:
   1%: -3.4942202045135513
   5%: -2.889485291005291
   10%: -2.5816762131519275

 PP Test Results:
   Test Statistic: -1.1578354109620839
   p-value: 0.6914659468592139
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.3700993760606226
   p-value: 0.09004337238766268
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [22]:
data = df['CD91일_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -2.3975058901896653
p-value: 0.1424415905303672
Critical Values:
   1%: -3.4942202045135513
   5%: -2.889485291005291
   10%: -2.5816762131519275

 PP Test Results:
   Test Statistic: -5.34735050831613
   p-value: 4.365419601738836e-06
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.6349467688108331
   p-value: 0.019459384653560625
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [23]:
data = df['CD91일_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -10.728539950065377
p-value: 3.0212677795320587e-19
Critical Values:
   1%: -3.4936021509366793
   5%: -2.8892174239808703
   10%: -2.58153320754717

 PP Test Results:
   Test Statistic: -20.411654482132068
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.09844359320539706
   p-value: 0.1
   Lags Used: 24
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [24]:
data = df['실업률']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.3833018389564644
p-value: 0.590254732221832
Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -3.655884200368189
   p-value: 0.004778254798030225
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.3126595331788241
   p-value: 0.1
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [25]:
data = df['실업률_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -4.482882180439881
p-value: 0.0002111290895832501


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -15.21178714998083
   p-value: 5.641305107976945e-28
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.18424422794524326
   p-value: 0.1
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [26]:
data = df['실업률_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.898493667278842
p-value: 2.8142713278927867e-07
Critical Values:
   1%: -3.4996365338407074
   5%: -2.8918307730370025
   10%: -2.5829283377617176

 PP Test Results:
   Test Statistic: -50.801599703209455
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.14541791023818784
   p-value: 0.1
   Lags Used: 17
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [27]:
data = df['로그동행지수순환변동치']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -2.8136494331809208
p-value: 0.05635034014672506
Critical Values:
   1%: -3.4968181663902103
   5%: -2.8906107514600103
   10%: -2.5822770483285953

 PP Test Results:
   Test Statistic: -2.0416633694115953
   p-value: 0.2686090661524686
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.13953079262053833
   p-value: 0.1
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [28]:
data = df['로그동행지수순환변동치_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -6.861485982214963
p-value: 1.5983012488879391e-09
Critical Values:
   1%: -3.4936021509366793
   5%: -2.8892174239808703
   10%: -2.58153320754717

 PP Test Results:
   Test Statistic: -5.219930537325945
   p-value: 8.004650578799595e-06
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.05798470449702616
   p-value: 0.1
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [29]:
data = df['로그동행지수순환변동치_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.705970268230385
p-value: 1.3013822091078573e-11
Critical Values:
   1%: -3.4968181663902103
   5%: -2.8906107514600103
   10%: -2.5822770483285953

 PP Test Results:
   Test Statistic: -18.95931472486719
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.018885930797465027
   p-value: 0.1
   Lags Used: 1
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [30]:
data = df['로그총지수']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: 1.0003497170237194
p-value: 0.9942696370981611
Critical Values:
   1%: -3.4989097606014496
   5%: -2.891516256916761
   10%: -2.5827604414827157

 PP Test Results:
   Test Statistic: 1.8948216199949206
   p-value: 0.9985196420322602
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 1.5229486896841433
   p-value: 0.01
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [31]:
data = df['로그총지수_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -2.147941011158405
p-value: 0.2256790521914679
Critical Values:
   1%: -3.4989097606014496
   5%: -2.891516256916761
   10%: -2.5827604414827157

 PP Test Results:
   Test Statistic: -8.040151813378419
   p-value: 1.856395088216361e-12
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.5287979790544735
   p-value: 0.035180635348091564
   Lags Used: 0
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [32]:
data = df['로그총지수_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -4.715823536546958
p-value: 7.857782033520365e-05
Critical Values:
   1%: -3.4989097606014496
   5%: -2.891516256916761
   10%: -2.5827604414827157

 PP Test Results:
   Test Statistic: -23.433529319369658
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.16720337507459535
   p-value: 0.1
   Lags Used: 34
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [33]:
data = df['로그KOSPI_증가']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -2.867048460870256
p-value: 0.049316186569592434
Critical Values:
   1%: -3.4968181663902103
   5%: -2.8906107514600103
   10%: -2.5822770483285953

 PP Test Results:
   Test Statistic: -2.0405078827100254
   p-value: 0.2690989551270685
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.8332621882533584
   p-value: 0.01
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [34]:
data = df['로그KOSPI_증가_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -10.804795415337345
p-value: 1.975832959475352e-19
Critical Values:
   1%: -3.4924012594942333
   5%: -2.8886968193364835
   10%: -2.5812552709190673

 PP Test Results:
   Test Statistic: -10.901215190767127
   p-value: 1.157973675358006e-19
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.09119296680374689
   p-value: 0.1
   Lags Used: 3
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [35]:
data = df['로그KOSPI_증가_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.352573888811266
p-value: 9.97052367758258e-11
Critical Values:
   1%: -3.4968181663902103
   5%: -2.8906107514600103
   10%: -2.5822770483285953

 PP Test Results:
   Test Statistic: -36.84428656516472
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.20284146929288185
   p-value: 0.1
   Lags Used: 24
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [36]:
df_filtered = df[~df['date'].isin(['2018/10', '2018/11', '2018/12'])]

df_filtered[['date', 'CD91일_2차차분', '실업률_2차차분', '로그동행지수순환변동치_2차차분', '로그총지수_2차차분', '로그KOSPI_증가_2차차분']].to_csv('독립변수.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('독립변수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>